# prj1 주제 : 쇼핑몰의 정보를 분석(평점,가격등)
## 타깃 : 네이버쇼핑
### 순서
- 1.크롤러제작
 

### 1. 크롤러 제작

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [4]:
query = input('제조사를 입력해주세요: ')
print(f"지금입력하신 제조사는 {query}입니다.")

titles = []
prices = []
review_counts = []
buy_counts = []
published_dates = []
favorites = []

driver = Chrome('./chromedriver')

for page_no in tqdm(range(1, 6)):

    page_url = f"https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
    driver.get(page_url)
    time.sleep(1)

    for scroll_down in range(7):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)

    list_basis = driver.find_element(By.CLASS_NAME,"list_basis")
    item_list = list_basis.find_elements(By.CLASS_NAME,'basicList_inner__xCM3J')

    items = []
    for i in tqdm(range(len(item_list))):
        item = item_list[i]
        title = item.find_element(By.CLASS_NAME,'basicList_title__VfX3c')
        titles.append(title.text)


        price = item.find_element(By.CLASS_NAME,"price_num__S2p_v").text[:-1].replace(',', '')
        prices.append(price)

        footer = item.find_element(By.CLASS_NAME,'basicList_etc_box__5lkgg')#.text
        reviews = footer.find_elements(By.CLASS_NAME,'basicList_num__sfz3h')
        footer_text = footer.text
        try:
            if "구매건수" in footer_text:
                review_counts.append(int(reviews[3].text.replace(',', '')))
                buy_counts.append(int(reviews[1].text.replace(',', '')))
                favorites.append(int(reviews[2].text.replace(',', '')))
                date = footer.find_elements(By.TAG_NAME,'span')[0].text[4:]
                published_dates.append(date)
            else:
                favorites.append(int(reviews[1].text.replace(',', '')))
                review_counts.append(int(reviews[0].text.replace(',', '')))
                date = footer.find_elements(By.CLASS_NAME,'basicList_etc__LSkN_')[1].text[4:]
                published_dates.append(date)

        except IndexError: ## 리뷰가 아직 충분하지 않아서, 정보가 안뜨는 케이스
            review_counts.append(0)
            favorites.append(int(reviews[0].text.replace(',', '')))
            date = footer.find_elements(By.TAG_NAME,'span')[0].text[4:]
            published_dates.append(date)
            
#길이 측정
print(len(titles), len(prices), len(review_counts), len(published_dates), len(favorites))

#df에담는다

result = pd.DataFrame({"제품명" : titles,
                       "가격" : prices,
                       "리뷰수" : review_counts,
                       "등록일" : published_dates,
                       "찜하기" : favorites})
result

#파일저장
result.to_excel(f"g:/dataAnal/dataset/{query}.xlsx", index=False)


지금입력하신 제조사는 샤오미입니다.


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

230 230 230 230 230


### 최종오류수정 및 적용완료!
- 오류파악완료 (태그네임과 클래스네임을 헷갈리지 말자!)
- 사용자의 입력을 받아서 크롤링하는 코드로 변경(생각보다 간단하다)
- formating으로 파일명 자동생성!
- Ending